<a href="https://colab.research.google.com/github/GuiXu40/deeplearning0/blob/main/Basic_code/Dlow%E6%BA%90%E7%A0%81%E5%88%86%E6%9E%90/001_%E6%95%B0%E6%8D%AE%E9%A2%84%E5%A4%84%E7%90%86%E4%BB%A3%E7%A0%81%E7%90%86%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

drive  sample_data


In [2]:
%cd drive/MyDrive/ColabNotebooks/DLow-master/

/content/drive/MyDrive/ColabNotebooks/DLow-master


In [3]:
!ls

data  LICENSE  models  motion_pred  README.md  results	Untitled0.ipynb  utils


In [4]:
!vim ./motion_pred/utils/dataset_h36m.py


7="./motion_pred/utils/dataset_h36m.py" 62L, 2646C▽  [>c]10;?]11;?import numpy as np
import os
from motion_pred.utils.dataset import Dataset
from motion_pred.utils.skeleton import Skeleton


class DatasetH36M(Dataset):def __init__(self, mode, t_his=25, t_pred=100, actions='all', use_vel=False)):self.use_vel = use_velsuper().__init__(mode, t_his, t_pred, actions)if use_vel:self.traj_dim += 3def prepare_data(self):self.data_file = os.path.join('data', 'data_3d_h36m.npz')self.subjects_split = {'train': [1, 5, 6, 7, 8],'test': [9, 11]}self.subjects = ['S%d' % x for x in self.subjects_split[self.mode]]self.skeleton = Skeleton(parents=[-1, 0, 1, 2, 3, 4, 0, 6, 7, 8, 9, 0, 111, 12, 13, 14, 12,16, 17, 18, 19, 20, 19, 22, 12, 24, 25@@@                                                                             1,1Top>8Vim: Caught deadly signal TERM

Vim: Finished.
^C


In [9]:
import numpy as np
h36 = np.load('./data/data_3d_h36m.npz',allow_pickle=True)

In [ ]:
h36.files
h36['positions_3d']

In [35]:
data_o = np.load('./data/data_3d_h36m.npz', allow_pickle=True)['positions_3d'].item()
#for k in data_o:
#  print(k)
# 每个文件夹 代表一个人， 这里只有7个，因为只有这7个人有3D 标签。

# 每个人的动作场景
#for k in list(dict(data_o['S1'].items()).keys()):
#  print(k)

#print(dict(data_o['S1'].items())['Eating'].shape)
#print(dict(data_o['S1'].items())['Eating'])

#dict(data_o['S1'].items())['Eating']
a = np.array([
    [1, 2, 3],
    [4, 5, 10],
    [7, 8, 9]
])
b = np.array([
    [1, 2],
    [4, 4],
    [7, 8]
])

print(a[:, 1:] - a[:, :1])
print(b[:, 1:] - b[:, :1])

[[1 2]
 [1 6]
 [1 2]]
[[1]
 [0]
 [1]]


### 1. dataset_h36m代码理解
该类继承自 dataset 类。

#### 1.1 dataset类代码

In [15]:
import numpy as np


class Dataset:

    def __init__(self, mode, t_his, t_pred, actions='all'):
        """
          mode: 训练模式或者测试模式
          t_his: 历史t时间步
          t_prep: 待预测的时间步
          actions: 具体的场景
        """
        self.mode = mode
        self.t_his = t_his
        self.t_pred = t_pred
        self.t_total = t_his + t_pred # 总时间步长
        self.actions = actions
        self.prepare_data() # 准备数据
        self.std, self.mean = None, None
        self.data_len = sum([seq.shape[0] for data_s in self.data.values() for seq in data_s.values()]) # data:[S*, 值] data_s[动作，值]
        self.traj_dim = (self.kept_joints.shape[0] - 1) * 3
        self.normalized = False
        # iterator specific
        self.sample_ind = None

    # 数据准备方法， 待实现
    def prepare_data(self):
        raise NotImplementedError

    # 数据归一化
    def normalize_data(self, mean=None, std=None):
        if mean is None:
            all_seq = []
            for data_s in self.data.values():
                for seq in data_s.values():
                    all_seq.append(seq[:, 1:])
            all_seq = np.concatenate(all_seq)
            self.mean = all_seq.mean(axis=0)
            self.std = all_seq.std(axis=0)
        else:
            self.mean = mean
            self.std = std
        for data_s in self.data.values():
            for action in data_s.keys():
                data_s[action][:, 1:] = (data_s[action][:, 1:] - self.mean) / self.std
        self.normalized = True

    # 随机采样 t_total 时间步长的动作序列
    def sample(self):
        subject = np.random.choice(self.subjects) # 生成随机数 subjects: s*
        dict_s = self.data[subject]
        action = np.random.choice(list(dict_s.keys()))
        seq = dict_s[action]
        fr_start = np.random.randint(seq.shape[0] - self.t_total)
        fr_end = fr_start + self.t_total
        traj = seq[fr_start: fr_end]
        return traj[None, ...]

    # dataloader batch_size=8, num_samples=1000: 采样总数
    def sampling_generator(self, num_samples=1000, batch_size=8):
        for i in range(num_samples // batch_size):
            sample = []
            for i in range(batch_size):
                sample_i = self.sample()
                sample.append(sample_i)
            sample = np.concatenate(sample, axis=0)
            yield sample # (b, T, 关节点数, 坐标维度)

    # 动作序列迭代器
    def iter_generator(self, step=25):
        for data_s in self.data.values():
            for seq in data_s.values():
                seq_len = seq.shape[0]
                for i in range(0, seq_len - self.t_total, step):
                    traj = seq[None, i: i + self.t_total]
                    yield traj





#### 1.2 Skeleton类

In [36]:
# Copyright (c) 2018-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import numpy as np


class Skeleton:
    def __init__(self, parents, joints_left, joints_right):
        assert len(joints_left) == len(joints_right)
        
        self._parents = np.array(parents)
        self._joints_left = joints_left
        self._joints_right = joints_right
        self._compute_metadata()
    
    # 返回关节点数
    def num_joints(self):
        return len(self._parents)
    
    # 返回父关节点
    def parents(self):
        return self._parents
    
    # 返回是否有子节点
    def has_children(self):
        return self._has_children

    # 返回子关节点
    def children(self):
        return self._children
    
    # 移除某些关节点
    def remove_joints(self, joints_to_remove):
        """
        Remove the joints specified in 'joints_to_remove'.
        """
        valid_joints = []

        # 保留不需要移除的关节点
        for joint in range(len(self._parents)):
            if joint not in joints_to_remove:
                valid_joints.append(joint)

        # 如果父节点被删除，就修改_parents，直到找到父节点
        for i in range(len(self._parents)):
            while self._parents[i] in joints_to_remove:
                self._parents[i] = self._parents[self._parents[i]]

        # 移除多余的空位       
        index_offsets = np.zeros(len(self._parents), dtype=int)
        new_parents = []
        for i, parent in enumerate(self._parents):
            if i not in joints_to_remove:
                new_parents.append(parent - index_offsets[parent])
            else:
                index_offsets[i:] += 1
        self._parents = np.array(new_parents)
        
        # 调整 左右关节点的位置
        if self._joints_left is not None:
            new_joints_left = []
            for joint in self._joints_left:
                if joint in valid_joints:
                    new_joints_left.append(joint - index_offsets[joint])
            self._joints_left = new_joints_left
        if self._joints_right is not None:
            new_joints_right = []
            for joint in self._joints_right:
                if joint in valid_joints:
                    new_joints_right.append(joint - index_offsets[joint])
            self._joints_right = new_joints_right

        self._compute_metadata()
        
        return valid_joints
    
    # 返回左关节点
    def joints_left(self):
        return self._joints_left
    
    def joints_right(self):
        return self._joints_right
        
    def _compute_metadata(self):
        self._has_children = np.zeros(len(self._parents)).astype(bool)
        for i, parent in enumerate(self._parents):
            if parent != -1:
                self._has_children[parent] = True

        self._children = []
        for i, parent in enumerate(self._parents):
            self._children.append([])
        for i, parent in enumerate(self._parents):
            if parent != -1:
                self._children[parent].append(i)

In [37]:
## 测试skeleton 类
skeleton = Skeleton(parents=[-1, 0, 1, 2, 3, 4, 0, 6, 7, 8, 9, 0, 11, 12, 13, 14, 12,
                                  16, 17, 18, 19, 20, 19, 22, 12, 24, 25, 26, 27, 28, 27, 30],
                          joints_left=[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23],
                          joints_right=[1, 2, 3, 4, 5, 24, 25, 26, 27, 28, 29, 30, 31])
removed_joints = {4, 5, 9, 10, 11, 16, 20, 21, 22, 23, 24, 28, 29, 30, 31}
kept_joints = np.array([x for x in range(32) if x not in removed_joints])
skeleton.remove_joints(removed_joints)

# 1.关节点数
print(skeleton.num_joints())
# 2.父节点
print(skeleton.parents())
# 3.是否有子节点
print(skeleton.has_children())
# 4.返回子节点
print(skeleton.children())
# 5.返回左关节点
print(skeleton.joints_left())
# 6.返回右关节点
print(skeleton.joints_right())

17
[-1  0  1  2  0  4  5  0  7  8  9  7 11 12  7 14 15]
[ True  True  True False  True  True False  True  True  True False  True
  True False  True  True False]
[[1, 4, 7], [2], [3], [], [5], [6], [], [8, 11, 14], [9], [10], [], [12], [13], [], [15], [16], []]
[4, 5, 6, 11, 12, 13]
[1, 2, 3, 14, 15, 16]


#### 1.3 dataset_h36m

In [ ]:
import numpy as np
import os
#from motion_pred.utils.dataset import Dataset
#from motion_pred.utils.skeleton import Skeleton


class DatasetH36M(Dataset):

    def __init__(self, mode, t_his=25, t_pred=100, actions='all', use_vel=False):
        self.use_vel = use_vel
        super().__init__(mode, t_his, t_pred, actions)
        if use_vel:
            self.traj_dim += 3

    # 从文件中读取数据
    def prepare_data(self):
        self.data_file = os.path.join('data', 'data_3d_h36m.npz')
        self.subjects_split = {'train': [1, 5, 6, 7, 8],
                               'test': [9, 11]}
        self.subjects = ['S%d' % x for x in self.subjects_split[self.mode]]
        self.skeleton = Skeleton(parents=[-1, 0, 1, 2, 3, 4, 0, 6, 7, 8, 9, 0, 11, 12, 13, 14, 12,
                                          16, 17, 18, 19, 20, 19, 22, 12, 24, 25, 26, 27, 28, 27, 30],
                                 joints_left=[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23],
                                 joints_right=[1, 2, 3, 4, 5, 24, 25, 26, 27, 28, 29, 30, 31])
        self.removed_joints = {4, 5, 9, 10, 11, 16, 20, 21, 22, 23, 24, 28, 29, 30, 31}
        self.kept_joints = np.array([x for x in range(32) if x not in self.removed_joints])
        self.skeleton.remove_joints(self.removed_joints)
        self.skeleton._parents[11] = 8
        self.skeleton._parents[14] = 8
        self.process_data()

    def process_data(self):
        data_o = np.load(self.data_file, allow_pickle=True)['positions_3d'].item()
        # 过滤出数据， filter(function, iterable) return list  然后 dict 转为字典
        data_f = dict(filter(lambda x: x[0] in self.subjects, data_o.items())) # 过滤出我们需要的数据 e.g. train:[1, 5, 6, 7, 8] 丢掉 9,11

        if self.actions != 'all':
            for key in list(data_f.keys()): # ['S1',...,'S8']
                data_f[key] = dict(filter(lambda x: all([a in x[0] for a in self.actions]), data_f[key].items())) # 筛选出需要的actions
                if len(data_f[key]) == 0: # 如果为0 说明没有该动作场景的动作，删除该人。
                    data_f.pop(key)
        for data_s in data_f.values(): # data_f.values(): [动作：数据]
            # 数据处理，最后一维的插值
            for action in data_s.keys(): # data_s.keys() : 动作
                seq = data_s[action][:, self.kept_joints, :] # data_s[action]: (2721, 32, 3)
                if self.use_vel:
                    v = (np.diff(seq[:, :1], axis=0) * 50).clip(-5.0, 5.0)
                    v = np.append(v, v[[-1]], axis=0)
                seq[:, 1:] -= seq[:, :1]
                if self.use_vel:
                    seq = np.concatenate((seq, v), axis=1)
                data_s[action] = seq
        self.data = data_f


if __name__ == '__main__':
    np.random.seed(0)
    actions = {'WalkDog'}
    dataset = DatasetH36M('train', actions=actions)
    generator = dataset.sampling_generator()
    dataset.normalize_data()
    # generator = dataset.iter_generator()
    for data in generator:
        print(data.shape)




### 总结
数据预处理部分，可以看出动作序列分为，S* 一共7个人，每一个人中有不同的动作序列，维度为(T, joints_num, joints_dim), 通过处理之后，可以返回(batch_size, T', joints_num', joints_dim) ,其中T'指的是我们需要的时间步长，包括历史序列，和对应的ground truth(未来序列)。joints_num'表示去除部分关节点之后的关节点数。